# bloom bytes: hugging face: 01 sentiment analysis
bb-hf-01-sa.ipynb

Notes
1. I'm using anaconda and created the environment as follows: <br>
 `conda create -n hugs python=3.9 tensorflow transformers pytorch jupyter sentencepiece`
2. Calling pipeline() will download the models needed.  Larger models may take some time to download.  pytorch is an especially large one (over 1 GB), and is needed for the summarization pipeline.  Models are stored in anaconda3/pkgs and are shared by all conda environments so don't need to download multiple times.  These can get big - my pkgs folder is over 20 GB.  Example: <br>
`Downloading pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]`
3. pipeline often returns warnings, typically along the following:
    - `Using a pipeline without specifying a model name and revision in production is not recommended.`
       <br>safe to ignore as these are just illustrative examples.  
    - `No model was supplied, defaulted to...`
       <br>safe to ignore as these are just illustrative examples.
    - ``Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.``
       <br> safe to ignore
    - `Some weights of the model checkpoint at... were not used when initializing...`
       <br> safe to ignore
    - ``ValueError: This tokenizer cannot be instantiated. Please make sure you have `sentencepiece` installed in order to use this tokenizer.``
       <br> errors like this typically occur when a module is missing.  With conda you can add with conda install X.
5. Hugging Face NLP Course link: https://huggingface.co/learn/nlp-course/chapter1/1?fw=pt
6. Hugging Face ModelHub: https://huggingface.co/docs/hub/en/models-the-hub
7. Hugging Face TextClassificationPipeline: https://huggingface.co/docs/transformers/v4.37.2/en/main_classes/pipelines#transformers.TextClassificationPipeline

In [6]:
import time
class timer:
    def __init__(self):
        self.t0 = time.time()
        self.t1 = 0
        self.t = -1
        self.state = "INIT"
    def start(self):
        if (self.state != "INIT") and (self.state != "STOP"):
            print("error - cannot start already started timer")
            return
        self.t0 = time.time()  # restart from now
        self.state = "START"
    def stopandprint(self):
        self.stop()
        self.print()
    def stop(self):
        if (self.state != "INIT") and (self.state != "START"):
            print("error - cannot stop already stopped timer")
            return
        self.t1 = time.time()
        self.t = self.t1 - self.t0
        self.state = "STOP"
        return self.t
    def print(self):
        if self.state == "STOP":
            print("Elapsed seconds: {:8.3f}".format(self.t))
        else:
            tmp = time.time() - self.t0
            print("Elapsed seconds: {:8.3f} (and counting)".format(tmp))

In [7]:
t = timer()
t.start()
time.sleep(1)
t.stop()
time.sleep(.5)
t.print()
t.start()
t.start()

Elapsed seconds:    1.005
error - cannot start already started timer


In [11]:
t.print()
t.stop()

Elapsed seconds:   26.635 (and counting)


26.63501000404358

# 1. Introducing pipeline and sentiment-analysis

In [12]:
from transformers import pipeline

In [13]:
classifier = pipeline(task="sentiment-analysis")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [14]:
classifier("this is a great book.  Everyone should read this book right now.")

[{'label': 'POSITIVE', 'score': 0.9998063445091248}]

In [15]:
classifier("this book is worse than any book ever")

[{'label': 'NEGATIVE', 'score': 0.9996964931488037}]

# M1

In [20]:
M1 = []
M1.append("This is a very medicore book. Read it if you like stories about talking dragons.")
M1.append("This is a very medicore book. Read it if you like silly stories about talking dragons.")
M1.append("This is a very medicore book. Read it if you like dumb stories about talking dragons.")
R1 = classifier(M1)

In [21]:
R1[0]

{'label': 'POSITIVE', 'score': 0.848868191242218}

In [22]:
R1[1]

{'label': 'POSITIVE', 'score': 0.5143406987190247}

In [23]:
R1[2]

{'label': 'NEGATIVE', 'score': 0.9953979849815369}

# M2

In [24]:
M2 = []
M2.append("This is not a novel to be tossed aside lightly.")
M2.append("This is not a novel to be tossed aside lightly. It should be thrown with great force.")
M2.append("This is not a novel to be tossed aside lightly. Don't even pick up this book in the first place.")
R2 = classifier(M2)

In [25]:
R2

[{'label': 'POSITIVE', 'score': 0.8818190693855286},
 {'label': 'POSITIVE', 'score': 0.9708155989646912},
 {'label': 'NEGATIVE', 'score': 0.992668867111206}]

# M3

In [26]:
t = timer()
M3 = []
M3.append("These aren't the droids you're looking for.")
M3.append("These are not the droids you're looking for.  These are, however, excellent droids")
M3.append("These are excellent droids, but they are not the droids you're looking for.")
M3.append("These are excellent droids.")
M3.append("These are great droids, almost what I wanted")
M3.append("These are great droids, but not what I wanted")
M3.append("These are great droids, but sometimes do their thing")
M3.append("These are great droids, but they constantly bicker")
M3.append("These are great droids, although they're very independent")
M3.append("These droids are good enough for government purposes, maybe not for enterprise use")
M3.append("These droids are good enough for government purposes, and for enterprise use")
R3 = classifier(M3)
t.stopandprint()

Elapsed seconds:    1.110


In [27]:
R3

[{'label': 'NEGATIVE', 'score': 0.9956250786781311},
 {'label': 'POSITIVE', 'score': 0.9992830157279968},
 {'label': 'NEGATIVE', 'score': 0.5603905916213989},
 {'label': 'POSITIVE', 'score': 0.9997205138206482},
 {'label': 'POSITIVE', 'score': 0.999283492565155},
 {'label': 'NEGATIVE', 'score': 0.9990296363830566},
 {'label': 'POSITIVE', 'score': 0.9945772886276245},
 {'label': 'NEGATIVE', 'score': 0.9469334483146667},
 {'label': 'POSITIVE', 'score': 0.9975984692573547},
 {'label': 'POSITIVE', 'score': 0.9303880929946899},
 {'label': 'POSITIVE', 'score': 0.9968749284744263}]

In [29]:
def process(texts, results, **kwargs):
    PROCESS_ARG_PRINT = "print"
    PROCESS_ARG_SORT = "sort"
    PROCESS_ARG_NORETURN = "noreturn"
    PROCESS_ARG_TRUNCATE = "truncate"
    
    printOpt = False
    sortOpt = False
    noreturnOpt = False
    truncateNum = -1
    
    if len(texts) != len(results):
        print("error - mismatched lists")
        return

    for key, value in kwargs.items():
        if key == PROCESS_ARG_PRINT:
            printOpt = kwargs[PROCESS_ARG_PRINT]
        elif key == PROCESS_ARG_SORT:
            sortOpt = kwargs[PROCESS_ARG_SORT]
        elif key == PROCESS_ARG_NORETURN:
            noreturnOpt = kwargs[PROCESS_ARG_NORETURN]
        elif key == PROCESS_ARG_TRUNCATE:
            truncateNum = kwargs[PROCESS_ARG_TRUNCATE]
        else:
            print("ignoring unrecognised arg", key)
      
    n = len(texts)
    Rn = []
    Rp = []
    R = []
    for i in range(n):
        confidence = results[i]["score"]
        sentiment = " "
        if results[i]["label"] == "NEGATIVE":
            sentiment = "-"
            if truncateNum > 0:
                Rn.append([confidence, sentiment, texts[i][0:truncateNum]])
            else:
                Rn.append([confidence, sentiment, texts[i]])
        else:
            sentiment = "+"
            if truncateNum > 0:
                Rp.append([confidence, sentiment, texts[i][0:truncateNum]])
            else:
                Rp.append([confidence, sentiment, texts[i]])
        
        if truncateNum > 0:
            R.append([confidence, sentiment, texts[i][0:truncateNum]])
        else:
            R.append([confidence, sentiment, texts[i]])
            
    if sortOpt:
        Rp = sorted(Rp, key = lambda x: -x[0])
        Rn = sorted(Rn, key = lambda x: x[0])
        R = Rp + Rn
    
    if printOpt:
        for r in R:
            print("{:3s} {:6.4f} {:s}".format(r[1], r[0], r[2]))    
    if noreturnOpt:
        return
    else:
        return R

In [30]:
process(M3, R3)

[[0.9956250786781311, '-', "These aren't the droids you're looking for."],
 [0.9992830157279968,
  '+',
  "These are not the droids you're looking for.  These are, however, excellent droids"],
 [0.5603905916213989,
  '-',
  "These are excellent droids, but they are not the droids you're looking for."],
 [0.9997205138206482, '+', 'These are excellent droids.'],
 [0.999283492565155, '+', 'These are great droids, almost what I wanted'],
 [0.9990296363830566, '-', 'These are great droids, but not what I wanted'],
 [0.9945772886276245,
  '+',
  'These are great droids, but sometimes do their thing'],
 [0.9469334483146667,
  '-',
  'These are great droids, but they constantly bicker'],
 [0.9975984692573547,
  '+',
  "These are great droids, although they're very independent"],
 [0.9303880929946899,
  '+',
  'These droids are good enough for government purposes, maybe not for enterprise use'],
 [0.9968749284744263,
  '+',
  'These droids are good enough for government purposes, and for enterp

In [31]:
process(M3, R3, print=True,noreturn=True)

-   0.9956 These aren't the droids you're looking for.
+   0.9993 These are not the droids you're looking for.  These are, however, excellent droids
-   0.5604 These are excellent droids, but they are not the droids you're looking for.
+   0.9997 These are excellent droids.
+   0.9993 These are great droids, almost what I wanted
-   0.9990 These are great droids, but not what I wanted
+   0.9946 These are great droids, but sometimes do their thing
-   0.9469 These are great droids, but they constantly bicker
+   0.9976 These are great droids, although they're very independent
+   0.9304 These droids are good enough for government purposes, maybe not for enterprise use
+   0.9969 These droids are good enough for government purposes, and for enterprise use


In [32]:
process(M3, R3, print=True,sort=True,noreturn=True)

+   0.9997 These are excellent droids.
+   0.9993 These are great droids, almost what I wanted
+   0.9993 These are not the droids you're looking for.  These are, however, excellent droids
+   0.9976 These are great droids, although they're very independent
+   0.9969 These droids are good enough for government purposes, and for enterprise use
+   0.9946 These are great droids, but sometimes do their thing
+   0.9304 These droids are good enough for government purposes, maybe not for enterprise use
-   0.5604 These are excellent droids, but they are not the droids you're looking for.
-   0.9469 These are great droids, but they constantly bicker
-   0.9956 These aren't the droids you're looking for.
-   0.9990 These are great droids, but not what I wanted


# M4

In [33]:
import os

fn = os.path.expanduser('~') + "/Dropbox/datasets/quotes-misc.txt"
print(fn)
with open(fn, "r") as f:
    M4 = f.readlines()
M4

/Users/jeremybloom/Dropbox/datasets/quotes-misc.txt


['# some quotes\n',
 'I enjoy tacos and burritos more than flautas.\n',
 'I enjoy tacos and burritos, but I dislike flautas.\n',
 'Please smoke the ham.\n',
 'Bro, do you even lift?\n',
 "The microwave functions great but it's a little louder than the one it replaces.  I like the color but not the price.\n",
 "The microwave functions better than expected for it's budget price\n",
 'You will be disappointed in this microwave.\n',
 'You will not be disappointed in this microwave.\n',
 'We should start a club.\n',
 'We should start a team.\n',
 'We should start a gang.\n',
 'We should start a private army.\n',
 'We should start an organization to take over the world.\n',
 'I enjoy margaritas almost as much as I enjoy surfing.\n',
 'If you pick up a starving dog and make him prosperous he will not bite you. This is the principal difference between a dog and man.\n',
 'It was a dark and stormy night\n',
 'I know why the caged bird sings\n',
 'Why, sometimes, I’ve believed as many as six imp

In [34]:
M4v2 = list(filter(lambda x: x[0] != "#", M4))  # remove commented lines

In [35]:
M4v2

['I enjoy tacos and burritos more than flautas.\n',
 'I enjoy tacos and burritos, but I dislike flautas.\n',
 'Please smoke the ham.\n',
 'Bro, do you even lift?\n',
 "The microwave functions great but it's a little louder than the one it replaces.  I like the color but not the price.\n",
 "The microwave functions better than expected for it's budget price\n",
 'You will be disappointed in this microwave.\n',
 'You will not be disappointed in this microwave.\n',
 'We should start a club.\n',
 'We should start a team.\n',
 'We should start a gang.\n',
 'We should start a private army.\n',
 'We should start an organization to take over the world.\n',
 'I enjoy margaritas almost as much as I enjoy surfing.\n',
 'If you pick up a starving dog and make him prosperous he will not bite you. This is the principal difference between a dog and man.\n',
 'It was a dark and stormy night\n',
 'I know why the caged bird sings\n',
 'Why, sometimes, I’ve believed as many as six impossible things befor

In [36]:
M4v2 = [s[:-1] for s in M4v2]                   # remove last char (which is a newline)

In [37]:
M4v2

['I enjoy tacos and burritos more than flautas.',
 'I enjoy tacos and burritos, but I dislike flautas.',
 'Please smoke the ham.',
 'Bro, do you even lift?',
 "The microwave functions great but it's a little louder than the one it replaces.  I like the color but not the price.",
 "The microwave functions better than expected for it's budget price",
 'You will be disappointed in this microwave.',
 'You will not be disappointed in this microwave.',
 'We should start a club.',
 'We should start a team.',
 'We should start a gang.',
 'We should start a private army.',
 'We should start an organization to take over the world.',
 'I enjoy margaritas almost as much as I enjoy surfing.',
 'If you pick up a starving dog and make him prosperous he will not bite you. This is the principal difference between a dog and man.',
 'It was a dark and stormy night',
 'I know why the caged bird sings',
 'Why, sometimes, I’ve believed as many as six impossible things before breakfast.',
 'Nobody realizes t

In [38]:
t.start()
R4v2 = classifier(M4v2)
t.stopandprint()

Elapsed seconds:    2.714


In [39]:
process(M4v2, R4v2, print=1, noreturn=1)

+   0.9996 I enjoy tacos and burritos more than flautas.
-   0.7773 I enjoy tacos and burritos, but I dislike flautas.
-   0.6885 Please smoke the ham.
-   0.7789 Bro, do you even lift?
+   0.5671 The microwave functions great but it's a little louder than the one it replaces.  I like the color but not the price.
+   0.9187 The microwave functions better than expected for it's budget price
-   0.9997 You will be disappointed in this microwave.
+   0.9975 You will not be disappointed in this microwave.
+   0.9886 We should start a club.
+   0.9918 We should start a team.
-   0.9947 We should start a gang.
-   0.9853 We should start a private army.
+   0.7775 We should start an organization to take over the world.
-   0.5903 I enjoy margaritas almost as much as I enjoy surfing.
+   0.9984 If you pick up a starving dog and make him prosperous he will not bite you. This is the principal difference between a dog and man.
-   0.7057 It was a dark and stormy night
+   0.9226 I know why the ca

In [40]:
process(M4v2, R4v2, print=1, sort=1, noreturn=1)

+   0.9996 I enjoy tacos and burritos more than flautas.
+   0.9989 The ultimate measure of a man is not where he stands in moments of comfort and convenience, but where he stands at times of challenge and controversy.
+   0.9984 If you pick up a starving dog and make him prosperous he will not bite you. This is the principal difference between a dog and man.
+   0.9975 You will not be disappointed in this microwave.
+   0.9922 I opened myself to the gentle indifference of the world.
+   0.9918 We should start a team.
+   0.9913 Man is nothing else but what he makes of himself
+   0.9886 We should start a club.
+   0.9453 Freedom is never voluntarily given by the oppressor; it must be demanded by the oppressed
+   0.9226 I know why the caged bird sings
+   0.9187 The microwave functions better than expected for it's budget price
+   0.7775 We should start an organization to take over the world.
+   0.6724 Where there is no hope, it is incumbent on us to invent it.
+   0.5671 The microw

# M5

In [41]:
import os
import pandas as pd
fn = os.path.expanduser('~') + "/Dropbox/datasets/amazon.csv"
df = pd.read_csv(fn)

In [42]:
df.head()

,product_id,product_name,category,discounted_price,actual_price,discount_percentage,rating,rating_count,about_product,user_id,user_name,review_id,review_title,review_content,img_link,product_link
0,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,Computers&Accessories|Accessories&Peripherals|...,₹399,"₹1,099",64%,4.2,"24,269",High Compatibility : Compatible With iPhone 12...,"AG3D6O4STAQKAY2UVGEUV46KN35Q,AHMY5CWJMMK5BJRBB...","Manav,Adarsh gupta,Sundeep,S.Sayeed Ahmed,jasp...","R3HXWT0LRP0NMF,R2AJM3LFTLZHFO,R6AQJGUP6P86,R1K...","Satisfied,Charging is really fast,Value for mo...",Looks durable Charging is fine tooNo complains...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Wayona-Braided-WN3LG1-Sy...
1,B098NS6PVG,Ambrane Unbreakable 60W / 3A Fast Charging 1.5...,Computers&Accessories|Accessories&Peripherals|...,₹199,₹349,43%,4.0,"43,994","Compatible with all Type C enabled devices, be...","AECPFYFQVRUWC3KGNLJIOREFP5LQ,AGYYVPDD7YG7FYNBX...","ArdKn,Nirbhay kumar,Sagar Viswanathan,Asp,Plac...","RGIQEG07R9HS2,R1SMWZQ86XIN8U,R2J3Y1WL29GWDE,RY...","A Good Braided Cable for Your Type C Device,Go...",I ordered this cable to connect my phone to An...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Ambrane-Unbreakable-Char...
2,B096MSW6CT,Sounce Fast Phone Charging Cable & Data Sync U...,Computers&Accessories|Accessories&Peripherals|...,₹199,"₹1,899",90%,3.9,"7,928",【 Fast Charger& Data Sync】-With built-in safet...,"AGU3BBQ2V2DDAMOAKGFAWDDQ6QHA,AESFLDV2PT363T2AQ...","Kunal,Himanshu,viswanath,sai niharka,saqib mal...","R3J3EQQ9TZI5ZJ,R3E7WBGK7ID0KV,RWU79XKQ6I1QF,R2...","Good speed for earlier versions,Good Product,W...","Not quite durable and sturdy,https://m.media-a...",https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Sounce-iPhone-Charging-C...
3,B08HDJ86NZ,boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...,Computers&Accessories|Accessories&Peripherals|...,₹329,₹699,53%,4.2,"94,363",The boAt Deuce USB 300 2 in 1 cable is compati...,"AEWAZDZZJLQUYVOVGBEUKSLXHQ5A,AG5HTSFRRE6NL3M5S...","Omkar dhale,JD,HEMALATHA,Ajwadh a.,amar singh ...","R3EEUZKKK9J36I,R3HJVYCLYOY554,REDECAZ7AMPQC,R1...","Good product,Good one,Nice,Really nice product...","Good product,long wire,Charges good,Nice,I bou...",https://m.media-amazon.com/images/I/41V5FtEWPk...,https://www.amazon.in/Deuce-300-Resistant-Tang...
4,B08CF3B7N1,Portronics Konnect L 1.2M Fast Charging 3A 8 P...,Computers&Accessories|Accessories&Peripherals|...,₹154,₹399,61%,4.2,"16,905",[CHARGE & SYNC FUNCTION]- This cable comes wit...,"AE3Q6KSUK5P75D5HFYHCRAOLODSA,AFUGIFH5ZAFXRDSZH...","rahuls6099,Swasat Borah,Ajay Wadke,Pranali,RVK...","R1BP4L2HH9TFUP,R16PVJEXKV6QZS,R2UPDB81N66T4P,R...","As good as original,Decent,Good one for second...","Bought this instead of original apple, does th...",https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Portronics-Konnect-POR-1...


In [43]:
df.loc[0]

product_id                                                    B07JW9H4J1
product_name           Wayona Nylon Braided USB to Lightning Fast Cha...
category               Computers&Accessories|Accessories&Peripherals|...
discounted_price                                                    ₹399
actual_price                                                      ₹1,099
discount_percentage                                                  64%
rating                                                               4.2
rating_count                                                      24,269
about_product          High Compatibility : Compatible With iPhone 12...
user_id                AG3D6O4STAQKAY2UVGEUV46KN35Q,AHMY5CWJMMK5BJRBB...
user_name              Manav,Adarsh gupta,Sundeep,S.Sayeed Ahmed,jasp...
review_id              R3HXWT0LRP0NMF,R2AJM3LFTLZHFO,R6AQJGUP6P86,R1K...
review_title           Satisfied,Charging is really fast,Value for mo...
review_content         Looks durable Charging is fi

In [44]:
df.shape

(1465, 16)

In [45]:
X = df["category"].unique()
print("len(X):", len(X))

len(X): 211


In [47]:
category = X[3]
print(category)

Electronics|HomeTheater,TV&Video|Televisions|SmartTelevisions


In [48]:
df2 = df[df["category"] == category]
df2.shape

(63, 16)

In [50]:
M5 = df2["review_content"].to_list()
len(M5)

63

In [51]:
t1 = timer()
R5 = classifier(M5)
t1.stopandprint()

Token indices sequence length is longer than the specified maximum sequence length for this model (1222 > 512). Running this sequence through the model will result in indexing errors


RuntimeError: The size of tensor a (1222) must match the size of tensor b (512) at non-singleton dimension 1

In [53]:
M5v2 = [m[:500] for m in M5]

In [54]:
len(M5v2)

63

In [55]:
M5v2 = list(set(M5v2))
len(M5v2)

37

In [56]:
t1 = timer()
R5v2 = classifier(M5v2)
t1.stopandprint()

Elapsed seconds:   76.273


In [57]:
process(M5v2, R5v2, truncate=200, print=True, sort=True, noreturn=True)

+   0.9998 Tv is good in this price range,It's an excellent product for this price range,Good,Picture quality is good,Amazing product sound quality is okay and smart features is little bit slow but it’s okay ove
+   0.9988 V nice and good,Good,Good,Very non responsive company,after several reminder to company and Amazon they don't bother to install the TV as committed since last 8 daysLet's see how they will respond in 
+   0.9988 Vu  UHD SMART 43"  web os  version good  tv 2022 , picture quality and opareting  nice  , but sound  quality I am not satisfied not afctvive  to other t vs,HDR can’t be turned off in apps,Got it at 19
+   0.9987 This OnePlus TV is great if you want a affordable and middle range features TV. I got it in the offer and it was totally worth it.Pros:-1. Sound is awesome - 100% satisfied with the quality of sound. 
+   0.9981 Update after 9 months of use:1. The TV is still going strong - all the good points mentioned in the original review below are still true and 

In [58]:
X = process(M5v2, R5v2, print=True, sort=True)

+   0.9998 Tv is good in this price range,It's an excellent product for this price range,Good,Picture quality is good,Amazing product sound quality is okay and smart features is little bit slow but it’s okay overall ✅ love this product,Ok super work,Good product,
+   0.9988 V nice and good,Good,Good,Very non responsive company,after several reminder to company and Amazon they don't bother to install the TV as committed since last 8 daysLet's see how they will respond in case of warranty or service requirement in future.,Excellent,Picture quality moderateSound quality moderateRemote access moderateAs per price product is Good.Product is Worth to buy .,Good tv with good service,Good
+   0.9988 Vu  UHD SMART 43"  web os  version good  tv 2022 , picture quality and opareting  nice  , but sound  quality I am not satisfied not afctvive  to other t vs,HDR can’t be turned off in apps,Got it at 19k in the sale. The TV uses WebOS. its snappy. its so so feature rich. The magic remote is amazing. 

In [59]:
X[0]

[0.9998489618301392,
 '+',
 "Tv is good in this price range,It's an excellent product for this price range,Good,Picture quality is good,Amazing product sound quality is okay and smart features is little bit slow but it’s okay overall ✅ love this product,Ok super work,Good product,"]

In [60]:
X[-1]

[0.9992063641548157,
 '-',
 'Hi all, firstly, I was very skeptical to purchase TV through online. Secondary, Iam a big fan of Sony and LG brands., Thirdly,as I have 4 year old kid at home I was so scared to spend money beyond 15k. Ofcourse at the first day sale for prime members when the Amazon great Indian festival started, I saw one Plus and Hisense A6H were priced at under 20k including bank offers and my favourite Sony 43inch was priced at 35k. But discarded Sony just bcz of expensive and scared if my kid whilst playing']

In [61]:
X[-3]

[0.9986599683761597,
 '-',
 "There's no magic remote and lg should include it by default as it makes life easier. The magic remote costs around 3k approximately.Display: when turned on tv and tested the display on black wallpaper found very small green and white dots.Immediately called up Amazon first and then we scheduled a tech visit.The tech did not come rather spoke on call and asked what's the problem and when informed about the problem he said needs to be replaced but no body came with it.So i waited for 2 days again "]